In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [2]:
data = []

for i in range(0, 34):
    url = 'https://www.jfklibrary.org/archives/other-resources/john-f-kennedy-speeches?page=' + str(i)
    page = requests.get(url).text
    soup = BeautifulSoup(page,"lxml")

    article_title = soup.select('h3.entry-title a')
    article_date = soup.select('div.time-stamp .date')

    for i in range(len(article_title)):
        data.append([datetime.today().strftime('%Y-%m-%d'), 
                     article_title[i].contents[0], 
                     'https://www.jfklibrary.org/' + article_title[i].get('href'), 
                     article_date[i].contents[0], '', ''])

In [3]:
df = pd.DataFrame(data, columns=['scrap_date', 'article_title', 'article_link', 'speech_date', 'speech_title', 'speech'])

df.head()

,scrap_date,article_title,article_link,speech_date,speech_title,speech
0,2020-11-19,Remarks of John F. Kennedy at an Induction Cer...,https://www.jfklibrary.org//archives/other-res...,"July 4, 1942",,
1,2020-11-19,"Remarks of John F. Kennedy, United War Fund Ap...",https://www.jfklibrary.org//archives/other-res...,"October 8, 1945",,
2,2020-11-19,Remarks of John F. Kennedy at the Crosscup-Pis...,https://www.jfklibrary.org//archives/other-res...,"November 11, 1945",,
3,2020-11-19,Remarks of John F. Kennedy at a Meeting of the...,https://www.jfklibrary.org//archives/other-res...,"April 15, 1946",,
4,2020-11-19,Remarks of John F. Kennedy on Radio Station WN...,https://www.jfklibrary.org//archives/other-res...,"May 16, 1946",,


In [7]:
for i in range(df.shape[0]):
    
    page = requests.get(df['article_link'][i]).text
    soup = BeautifulSoup(page,"lxml")

    if i == 0:
        speech_title = soup.select('.field h1 span')
        article_ = soup.select('.field p span span span span span span')
    else:
        speech_title = soup.select('article.entry-content > div.field > h1')
        article_ = soup.select('article.entry-content > div.field > p span span')
    
    if len(speech_title) != 0:
        df.at[i,'speech_title'] = speech_title[0].contents[0]
    else:
        df.at[i,'speech_title'] =  ''
    
    article_text = ''
    for text_ in article_:
        if not str(text_.contents[0]).startswith('<'):
            article_text = article_text + text_.contents[0]
    
    df.at[i,'speech'] = article_text

In [9]:
df.head()

,scrap_date,article_title,article_link,speech_date,speech_title,speech
0,2020-11-19,Remarks of John F. Kennedy at an Induction Cer...,https://www.jfklibrary.org//archives/other-res...,"July 4, 1942",FOR WHAT WE FIGHT,A number of years ago in a room in Philadelphi...
1,2020-11-19,"Remarks of John F. Kennedy, United War Fund Ap...",https://www.jfklibrary.org//archives/other-res...,"October 8, 1945",,
2,2020-11-19,Remarks of John F. Kennedy at the Crosscup-Pis...,https://www.jfklibrary.org//archives/other-res...,"November 11, 1945",,
3,2020-11-19,Remarks of John F. Kennedy at a Meeting of the...,https://www.jfklibrary.org//archives/other-res...,"April 15, 1946",Public Responsibility for the Veteran,
4,2020-11-19,Remarks of John F. Kennedy on Radio Station WN...,https://www.jfklibrary.org//archives/other-res...,"May 16, 1946",THE VETERAN,


In [32]:
df.to_csv('JFK_speech.csv', index=False)